# Benchmarking skforecast

This notebook benchmarks the performance (velocity) of the `skforecast` in its different versions and keeps track of the results.

**Notes**

+ In version `0.15.0` the binning of residuals was introduced in multi-series forecasters. This explains the increase in the time taken to fit the model.

In [8]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast


In [9]:
# !pip install --upgrade numpy < 2.0 && \
# pip install --upgrade pandas && \
# pip install --upgrade scikit-learn

In [10]:
# !pip install skforecast==0.15.1
# !pip install skforecast==0.14.0
# !pip install skforecast==0.13.0

In [11]:
# Libraries
# ==============================================================================
from benchmarking import BenchmarkRunner, plot_benchmark_results
import skforecast
import platform
import psutil
import plotly.express as px
if skforecast.__version__ >= '0.14.0':
    from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries
    from skforecast.direct import ForecasterDirect, ForecasterDirectMultiVariate
    from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster, backtesting_forecaster_multiseries
else:
    from skforecast.ForecasterAutoreg import ForecasterAutoreg
    from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
    from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
    from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
    from skforecast.model_selection import backtesting_forecaster, backtesting_forecaster_multiseries
from skforecast.utils import *
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

In [12]:
print(f"Python version: {platform.python_version()}")
print(f"skforecast version: {skforecast.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"Computer network name: {platform.node()}")
print(f"Processor type: {platform.processor()}")
print(f"Platform type: {platform.platform()}")
print(f"Operating system: {platform.system()}")
print(f"Operating system release: {platform.release()}")
print(f"Operating system version: {platform.version()}")
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")

Python version: 3.12.9
skforecast version: 0.16.0
scikit-learn version: 1.6.1
pandas version: 2.2.3
numpy version: 2.2.5
Computer network name: joaquin-HP-ProBook-440-G6
Processor type: x86_64
Platform type: Linux-6.11.0-21-generic-x86_64-with-glibc2.39
Operating system: Linux
Operating system release: 6.11.0-21-generic
Operating system version: #21~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Feb 24 16:52:15 UTC 2
Number of physical cores: 4
Number of logical cores: 8


# ForecasterRecursiveMultiSeries

In [13]:
# Mock data for benchmarking
# ==========================================================
n_series = 600
len_series = 2000
series_dict = {}
rng = np.random.default_rng(321)
for i in range(n_series):
    series_dict[f'series_{i}'] = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name=f'series_{i}'
    )
series_dataframe = pd.DataFrame(series_dict, index=series_dict['series_0'].index)

exog_dict = {}
rng = np.random.default_rng(321)
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=series_dict[k].index
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict[k] = exog


exog_dict_prediction = {}
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=pd.date_range(
                start=series_dict[k].index.max() + pd.Timedelta(hours=1),
                periods=100,
                freq='h'
            )
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict_prediction[k] = exog

In [ ]:
# Benchmarking speed ForecasterRecursiveMultiSeries
# ==============================================================================
regressor = LGBMRegressor(random_state=8520, verbose=-1)
# regressor = LinearRegression()
if skforecast.__version__ >= '0.14.0':
    forecaster = ForecasterRecursiveMultiSeries(
        regressor=regressor,
        lags=50,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        encoding="ordinal"
    )
else:
     forecaster = ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=50,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        encoding="ordinal"
    )

def ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict():
    forecaster.fit(series=series_dict, exog=exog_dict)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict():
    forecaster._create_train_X_y(series=series_dict, exog=exog_dict)

def ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog():
    forecaster.fit(series=series_dataframe)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog():
    forecaster._create_train_X_y(series=series_dataframe)

def ForecasterRecursiveMultiSeries__create_train_X_y_single_series():
    _ = forecaster._create_train_X_y_single_series(
            y = series_dict['series_0'],
            exog = exog_dict['series_0'],
            ignore_exog = False,
        )

def ForecasterRecursiveMultiSeries_predict_exog_is_dict():
    forecaster.predict(steps=100, exog=exog_dict_prediction, suppress_warnings=True)

def ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict():
    _ = forecaster._create_predict_inputs(
            steps         = 100,
            exog         = exog_dict_prediction,
            check_inputs = True
        )

def ForecasterRecursiveMultiSeries__check_predict_inputs():
    check_predict_input(
        forecaster_name  = type(forecaster).__name__,
        steps            = 100,
        is_fitted        = forecaster.is_fitted,
        exog_in_         = forecaster.exog_in_,
        index_type_      = forecaster.index_type_,
        index_freq_      = forecaster.index_freq_,
        window_size      = forecaster.window_size,
        last_window      = pd.DataFrame(forecaster.last_window_),
        exog             = exog_dict_prediction,
        exog_type_in_    = forecaster.exog_type_in_,
        exog_names_in_   = forecaster.exog_names_in_,
        interval         = None,
        levels           = forecaster.series_names_in_,
        series_names_in_ = forecaster.series_names_in_,
        encoding         = forecaster.encoding
    )

def ForecasterRecursiveMultiSeries_backtesting_series_is_dict_exog_is_dict():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dict,
                exog=exog_dict,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dict,
                exog=exog_dict,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )
        
def ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_no_exog():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )


runner = BenchmarkRunner(repeat=5, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog, forecaster= forecaster)
runner = BenchmarkRunner(repeat=10, output_dir="./")
forecaster.fit(series=series_dict, exog=exog_dict)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_single_series, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_predict_exog_is_dict, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__check_predict_inputs, forecaster= forecaster)
runner = BenchmarkRunner(repeat=5, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dict_exog_is_dict, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_no_exog, forecaster= forecaster)

In [17]:
# Plot results
# ==============================================================================
results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark = results_benchmark_all.query("forecaster_name in ['ForecasterRecursiveMultiSeries', 'ForecasterAutoregMultiSeries']").copy()
results_benchmark = results_benchmark.query("regressor_name == 'LGBMRegressor'").copy()
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'").copy()
    display(df)
    plot_benchmark_results(df, function_name)

,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
0,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,12.293677,0.429946,2025-04-23 15:03:26.238942,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
12,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,12.290554,0.785076,2025-04-23 15:09:20.254703,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
24,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,11.181269,0.577565,2025-04-23 15:13:23.496449,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
36,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,12.332075,1.001781,2025-04-23 15:25:24.855452,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
48,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,12.542493,0.606813,2025-04-23 15:34:44.825479,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
68,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,10.987336,0.396632,2025-04-23 15:54:09.384976,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
80,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,11.761212,0.358871,2025-04-23 15:58:20.920219,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
92,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,11.800770,0.402881,2025-04-23 16:02:51.964754,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
104,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,11.717140,0.238003,2025-04-23 16:06:12.882601,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
116,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,11.314107,0.397571,2025-04-23 16:09:47.625542,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
1,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.594639,0.061033,2025-04-23 15:04:27.710376,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
13,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.730657,0.036684,2025-04-23 15:10:21.711978,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
25,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.008426,0.020794,2025-04-23 15:14:19.407835,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
37,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.624174,0.090886,2025-04-23 15:26:26.520388,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
49,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.603322,0.046577,2025-04-23 15:35:47.542332,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
69,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.998299,0.020942,2025-04-23 15:55:04.326742,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
81,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.760219,0.029486,2025-04-23 15:59:19.730993,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
93,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.708451,0.049639,2025-04-23 16:03:50.974931,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
105,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.560971,0.057417,2025-04-23 16:07:11.473435,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
117,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.534163,0.040951,2025-04-23 16:10:44.200979,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
2,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,11.580742,0.917493,2025-04-23 15:04:35.688064,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
14,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,10.828553,0.107749,2025-04-23 15:10:30.369368,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
26,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,10.099889,0.160311,2025-04-23 15:14:29.454622,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
38,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,11.003430,0.223845,2025-04-23 15:26:39.645552,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
50,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,11.890319,0.521044,2025-04-23 15:36:00.563508,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
70,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,10.926485,0.812523,2025-04-23 15:55:14.322641,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
82,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,11.199184,0.096970,2025-04-23 15:59:28.536409,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
94,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,11.031137,0.134091,2025-04-23 16:03:59.522090,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
106,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,11.330288,0.895549,2025-04-23 16:07:19.282706,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
118,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,10.856502,0.081189,2025-04-23 16:10:51.876257,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
3,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.224587,0.055873,2025-04-23 15:05:33.595536,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
15,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.375285,0.040809,2025-04-23 15:11:24.516068,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
27,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.517815,0.064871,2025-04-23 15:15:19.958170,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
39,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,2.004793,0.040390,2025-04-23 15:27:34.667213,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
51,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,2.146876,0.070924,2025-04-23 15:37:00.019066,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
71,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.568071,0.119750,2025-04-23 15:56:08.959300,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
83,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.411051,0.055234,2025-04-23 16:00:24.536982,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
95,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.385172,0.047952,2025-04-23 16:04:54.682880,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
107,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.270865,0.041491,2025-04-23 16:08:15.939377,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
119,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.206709,0.044674,2025-04-23 16:11:46.163407,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
4,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001306,0.000172,2025-04-23 15:05:51.677126,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
16,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001370,0.000224,2025-04-23 15:11:43.049817,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
28,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001605,0.000359,2025-04-23 15:15:38.790638,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
40,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001624,0.000139,2025-04-23 15:27:57.642580,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
52,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.002253,0.000301,2025-04-23 15:37:24.534882,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
72,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.003732,0.000383,2025-04-23 15:56:31.510600,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
84,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001514,0.000252,2025-04-23 16:00:43.539847,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
96,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001539,0.000270,2025-04-23 16:05:13.551260,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
108,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001319,0.000149,2025-04-23 16:08:35.307922,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
120,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001437,0.000155,2025-04-23 16:12:03.939259,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
5,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,0.451472,0.038231,2025-04-23 15:05:51.687220,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
17,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,1.681694,0.031587,2025-04-23 15:11:43.060889,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
29,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,1.632832,0.007875,2025-04-23 15:15:38.802105,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
41,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,42.354621,1.205520,2025-04-23 15:27:57.654222,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
53,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,43.534917,2.283648,2025-04-23 15:37:24.551023,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
73,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,1.796171,0.151445,2025-04-23 15:56:31.536580,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
85,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,1.703143,0.009877,2025-04-23 16:00:43.551471,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
97,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,1.695652,0.028556,2025-04-23 16:05:13.562818,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
109,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,0.463045,0.041767,2025-04-23 16:08:35.319060,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
121,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,0.461218,0.038811,2025-04-23 16:12:03.950433,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
6,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,0.294342,0.032782,2025-04-23 15:05:53.948491,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
18,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,1.551544,0.048945,2025-04-23 15:11:51.473073,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
30,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,1.508526,0.016043,2025-04-23 15:15:46.970652,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
42,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,NaN,NaN,2025-04-23 15:31:29.431385,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
54,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,NaN,NaN,2025-04-23 15:41:02.231555,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
74,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,1.582764,0.096881,2025-04-23 15:56:40.522196,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
86,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,1.705099,0.281979,2025-04-23 16:00:52.071181,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
98,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,1.547401,0.029990,2025-04-23 16:05:22.045873,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
110,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,0.300234,0.034957,2025-04-23 16:08:37.638061,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
122,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,0.306396,0.038716,2025-04-23 16:12:06.261105,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
7,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.075259,0.011461,2025-04-23 15:05:55.423850,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
19,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.285421,0.006976,2025-04-23 15:11:59.234735,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
31,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.292057,0.017751,2025-04-23 15:15:54.517700,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
43,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,NaN,NaN,2025-04-23 15:31:29.435634,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
55,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,NaN,NaN,2025-04-23 15:41:02.236475,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
75,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.320222,0.009129,2025-04-23 15:56:48.439766,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
87,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.422475,0.069229,2025-04-23 16:01:00.604398,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
99,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.288091,0.006007,2025-04-23 16:05:29.787076,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
111,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.068207,0.000755,2025-04-23 16:08:39.143474,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
123,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.078871,0.014604,2025-04-23 16:12:07.797771,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
128,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,81259b5066e4c039ab1805445e26e639,NaN,NaN,2025-04-23 20:07:35.146022,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
133,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,045a2922d13f7327fc5635f0fa1d968e,NaN,NaN,2025-04-23 20:08:07.633611,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
134,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,5c9a4d89a695fe2ded0327ec81495237,19.799778,1.440243,2025-04-23 20:08:27.626583,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
143,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,21928d6bc9058a601b52a861e58a4cfb,22.981285,1.349951,2025-04-23 20:13:13.189265,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
156,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,21928d6bc9058a601b52a861e58a4cfb,38.222740,0.253136,2025-04-23 20:19:24.990623,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
169,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,21928d6bc9058a601b52a861e58a4cfb,43.853923,3.686905,2025-04-23 22:00:26.938013,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
182,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,21928d6bc9058a601b52a861e58a4cfb,42.174608,1.797158,2025-04-23 22:13:33.028822,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
195,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,764622d682985b184e7b2369791c61a1,41.622226,2.192564,2025-04-23 22:47:46.896636,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
209,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,764622d682985b184e7b2369791c61a1,21.979430,0.147381,2025-04-23 22:56:07.201614,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
210,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,14.9991,0.478435,2025-04-23 22:57:57.105248,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


# ForecasterRecursive

In [ ]:
# Mock data for benchmarking
# ==========================================================
len_series = 2000
rng = np.random.default_rng(321)
y = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name='y'
    )
rng = np.random.default_rng(321)
exog = pd.DataFrame(index=y.index)
exog['day_of_week'] = exog.index.dayofweek
exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
exog['month'] = exog.index.month
exog_prediction = pd.DataFrame(
                    index=pd.date_range(
                        start=series_dict[k].index.max() + pd.Timedelta(hours=1),
                        periods=100,
                        freq='h'
                    )
                 ) 
exog_prediction['day_of_week'] = exog_prediction.index.dayofweek
exog_prediction['week_of_year'] = exog_prediction.index.isocalendar().week.astype(int)
exog_prediction['month'] = exog_prediction.index.month

In [11]:
# Benchmarking speed ForecasterRecursive
# ==============================================================================
if skforecast.__version__ >= '0.14.0':
    forecaster = ForecasterRecursive(
        regressor=regressor,
        lags=50,
        transformer_y=StandardScaler(),
        transformer_exog=StandardScaler(),
    )
else:
     forecaster = ForecasterAutoreg(
        regressor=regressor,
        lags=50,
        transformer_y=StandardScaler(),
        transformer_exog=StandardScaler(),
    )

def ForecasterRecursive_fit():
    forecaster.fit(y=y, exog=exog)

def ForecasterRecursive__create_train_X_y():
    if skforecast.__version__ >= '0.14.0':
        forecaster._create_train_X_y(y=y, exog=exog)
    else:
        forecaster.create_train_X_y(y=y, exog=exog)


def ForecasterRecursive_predict():
    forecaster.predict(steps=100, exog=exog_prediction)

def ForecasterRecursive__create_predict_inputs():
    _ = forecaster._create_predict_inputs(
            steps        = 100,
            exog         = exog_prediction,
            check_inputs = True
        )
    
def ForecasterRecursive_backtesting():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )


runner = BenchmarkRunner(repeat=10, output_dir="./")
_ = runner.benchmark(ForecasterRecursive_fit, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursive__create_train_X_y, forecaster= forecaster)
forecaster.fit(y=y, exog=exog)
_ = runner.benchmark(ForecasterRecursive_predict, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursive__create_predict_inputs, forecaster= forecaster)

runner = BenchmarkRunner(repeat=5, output_dir="./")
_ = runner.benchmark(ForecasterRecursive_backtesting, forecaster= forecaster)

Benchmarking function: ForecasterRecursive_fit with skforecast version 0.16.0
Benchmarking function: ForecasterRecursive__create_train_X_y with skforecast version 0.16.0
Benchmarking function: ForecasterRecursive_predict with skforecast version 0.16.0
Benchmarking function: ForecasterRecursive__create_predict_inputs with skforecast version 0.16.0
Benchmarking function: ForecasterRecursive_backtesting with skforecast version 0.16.0


In [18]:
# Plot results
# ==============================================================================
results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark = results_benchmark_all.query("forecaster_name in ['ForecasterRecursive', 'ForecasterAutoreg']").copy()
results_benchmark = results_benchmark.query("regressor_name == 'LGBMRegressor'").copy()
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'").copy()
    display(df)
    plot_benchmark_results(df, function_name)

,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
8,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.200046,0.021582,2025-04-23 15:05:56.608429,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
20,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.191685,0.046102,2025-04-23 15:12:01.602347,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
32,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.201174,0.040401,2025-04-23 15:15:57.129152,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
44,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.236430,0.039534,2025-04-23 15:31:29.965594,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
56,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.255430,0.060431,2025-04-23 15:41:03.640028,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
60,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.217465,0.029841,2025-04-23 15:46:05.440298,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
64,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.210943,0.030716,2025-04-23 15:46:38.741227,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
76,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.275427,0.034657,2025-04-23 15:56:50.851427,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
88,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.274136,0.074695,2025-04-23 16:01:03.371465,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
100,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.193109,0.022947,2025-04-23 16:05:31.689260,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
9,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,0.003582,0.000336,2025-04-23 15:05:58.613440,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
21,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,0.002786,0.000231,2025-04-23 15:12:03.522895,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
33,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,0.003637,0.000685,2025-04-23 15:15:59.144846,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
45,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,NaN,NaN,2025-04-23 15:31:32.335009,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
57,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,NaN,NaN,2025-04-23 15:41:06.200461,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
61,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive__create_train_X_y,1302d78676d713cc5e64125ac23a65d2,NaN,NaN,2025-04-23 15:46:07.619566,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
65,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive__create_train_X_y,20bdfc9de1cb7a9e99e703c087d136d3,0.003645,0.000583,2025-04-23 15:46:40.854571,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
77,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,20bdfc9de1cb7a9e99e703c087d136d3,0.004178,0.000579,2025-04-23 15:56:53.610990,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
89,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,20bdfc9de1cb7a9e99e703c087d136d3,0.003653,0.000277,2025-04-23 16:01:06.118167,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
101,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,20bdfc9de1cb7a9e99e703c087d136d3,0.002773,0.000379,2025-04-23 16:05:33.624338,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
10,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.075136,0.004202,2025-04-23 15:05:58.891908,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
22,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.058053,0.001119,2025-04-23 15:12:03.760460,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
34,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.084377,0.019527,2025-04-23 15:15:59.361690,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
46,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.075075,0.003942,2025-04-23 15:31:32.543497,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
58,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.080083,0.004092,2025-04-23 15:41:06.477888,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
62,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.068110,0.005326,2025-04-23 15:46:07.814583,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
66,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.068882,0.004036,2025-04-23 15:46:41.087891,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
78,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.089361,0.006090,2025-04-23 15:56:53.918733,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
90,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.077495,0.007502,2025-04-23 16:01:06.363095,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
102,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.061347,0.002380,2025-04-23 16:05:33.829933,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
11,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,0.002528,0.000266,2025-04-23 15:05:59.648667,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
23,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,0.002659,0.000322,2025-04-23 15:12:04.344930,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
35,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,0.003698,0.000622,2025-04-23 15:16:00.211718,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
47,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,NaN,NaN,2025-04-23 15:31:33.298997,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
59,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,NaN,NaN,2025-04-23 15:41:07.284418,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
63,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,NaN,NaN,2025-04-23 15:46:08.499926,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
67,ForecasterAutoreg,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,NaN,NaN,2025-04-23 15:46:41.780575,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
79,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,0.003530,0.000365,2025-04-23 15:56:54.817583,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
91,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,0.003004,0.000305,2025-04-23 16:01:07.143048,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
103,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,0.002960,0.000349,2025-04-23 16:05:34.448297,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
200,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_backtesting,dad167768fbdc53eca10d293c5861956,0.920042,0.154116,2025-04-23 22:51:18.980518,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
215,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_backtesting,dad167768fbdc53eca10d293c5861956,0.823157,0.042117,2025-04-23 22:59:16.873858,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


# Summary of historical results

In [ ]:
results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark_all['function_name'] = results_benchmark_all['function_name'].str.split('_', n=1).str[1]
results_benchmark_all = results_benchmark_all.groupby(['forecaster_name', 'skforecast_version', 'function_name'])['run_time_avg'].agg('median').reset_index()
results_benchmark_all = results_benchmark_all.sort_values(by=['function_name'])

fig = px.bar(
    results_benchmark_all.query("forecaster_name in ['ForecasterRecursiveMultiSeries', 'ForecasterAutoregMultiSeries']"),
    x='function_name',
    y='run_time_avg',
    color='skforecast_version',
    barmode='group',
    title='MultiSeries Forecasters - Median Run Time by Function and skforecast Version',
    labels={'run_time_avg': 'Median Run Time (s)', 'function_name': 'Function'},
    category_orders={'skforecast_version': sorted(results_benchmark_all['skforecast_version'].unique())}
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

fig = px.bar(
    results_benchmark_all.query("forecaster_name in ['ForecasterRecursive', 'ForecasterAutoreg']"),
    x='function_name',
    y='run_time_avg',
    color='skforecast_version',
    barmode='group',
    title='Single-Series Forecasters - Median Run Time by Function and skforecast Version',
    labels={'run_time_avg': 'Median Run Time (s)', 'function_name': 'Function'},
    category_orders={'skforecast_version': sorted(results_benchmark_all['skforecast_version'].unique())}
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()
